<h1>HW4 - Attention and Transformers- Practical Q2</h1>
<h3><font color=yellow>Total Points: 100</font></h3>



<font size=+4>
Introduction
</font>
<br/>
<font size=+2>

<br/>
Part-of-speech tagging (POS tagging) is a natural language processing task where each word in a given text is assigned a specific part-of-speech category, such as noun, verb, adjective, etc. In the context of Roberta, a state-of-the-art language model based on the transformer architecture, POS tagging involves leveraging the model's pre-trained knowledge to accurately predict and label the grammatical roles of words in a sentence. Roberta's extensive training data and attention mechanisms enable it to capture subtle contextual cues, allowing for more nuanced and accurate POS tagging. By understanding the syntactic structure of a sentence, Roberta enhances the efficiency and precision of POS tagging, contributing to more sophisticated language understanding and downstream applications in natural language processing.
</font>


<font size=+2>
In this section we want to fine-tune the Roberta that you trained in previous section for POS-Tagging Task on Parsig dataset
<font/>

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

import re
import torch
import pandas as pd

import numpy as np
import transformers
import matplotlib.pyplot as plt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
# All label that exist
# AAX mean Unknown used for CLS and other symbols
tags = {'INIT','AAX', 'B-ADJ', 'I-ADJ', 'B-ADV', 'I-ADV', 'B-N', 'I-N', 'B-V', 'I-V',\
        'B-PRONOUN', 'I-PRONOUN', 'B-NUM', 'I-NUM', 'B-DET', 'I-DET', 'B-PRE',\
        'I-PRE', 'B-POST', 'I-POST', 'B-CONJ', 'I-CONJ', 'B-JUNK', 'I-JUNK',\
        'B-MARKER', 'I-MARKER','FINISH','PAD'}
# Convert lables to speceif numeric id
tag2id = {tag: id for id, tag in enumerate(sorted(list(tags)))}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
######################   TODO 1.1   ########################
# Load pretrained model that trained in previous sectiopn
# Load tokenizer from previous section
# Freeze base model for fine-tuning

from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("fazly1380/mlm_tokenizer")
pretrained_model = AutoModelForMaskedLM.from_pretrained("fazly1380/mlm")


for param in pretrained_model.parameters():
    param.requires_grad = False

###################### (5 points) ##########################


tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/65.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/291k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def load_data(file_name):
    df = pd.read_csv(file_name)
    for index, row in df.iterrows():
        tags = row['labels'].split(',')
        df.at[index, 'labels'] = [tag2id[tag] for tag in tags]
    return df

train_df = load_data("train_df.csv")
test_df = load_data("test_df.csv")

train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=21)



In [ ]:
######################   TODO 1.3   ########################
# Complete custom data set
# Use Torch dataloader for datasets
###################### (15 points) ##########################
######################   TODO 1.3   ########################
# Complete custom data set
# Use Torch dataloader for datasets
###################### (15 points) ##########################

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import random

max_len = 40

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.texts = df['text'].tolist()
        self.labels = df['labels'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, index):
        text = self.texts[index]
        labels = self.labels[index]
        labels = [0] +labels+[26]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',  # Pad to the maximum length
            max_length=self.max_length,
            return_tensors='pt'
        )




        if len(labels) > self.max_length:

            dif = len(labels) - self.max_length
            init_index = random.randint(0,dif-1)
            labels = labels[init_index:init_index + self.max_length]

        else:
            labels.extend([27] * (self.max_length - len(labels)))
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(labels, dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)


train = CustomDataset(train_df, tokenizer, max_len)
test = CustomDataset(test_df, tokenizer, max_len)
validation = CustomDataset(valid_df, tokenizer, max_len)

batch_size = 64
trainloader = DataLoader(train, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test, batch_size=batch_size, shuffle=True)
validloader = DataLoader(validation, batch_size=batch_size, shuffle=True)

In [ ]:
######################   TODO 2.1   ########################
# Write a code for training model with training datset
# At each epoch report accurcy for validation dataset
# Save best model by accuracy
###################### (30 points) ##########################

import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
class POS(nn.Module):
    def __init__(self, input_size, num_tags, hidden):
        super(POS, self).__init__()
        self.fc = nn.Linear(input_size, hidden)
        self.tanh = nn.Tanh()
        #self.bn1 = nn.BatchNorm1d(hidden)
        self.dropout = nn.Dropout(0.1)
        self.fc_2 = nn.Linear(hidden, num_tags)
        #self.bn2 = nn.BatchNorm1d(num_tags)

    def forward(self, x):
        x = self.fc(x)
        x = self.tanh(x)
        #x = self.bn1(x)
        x = self.dropout(x)
        x = self.fc_2(x)
        #x = self.bn2(x)
        return x

pos_head = POS(pretrained_model.config.hidden_size, len(tags), 512)

class ModifiedCombinedModel(nn.Module):
    def __init__(self, roberta_model, pos_head, additional_layer=None):
        super(ModifiedCombinedModel, self).__init__()
        self.roberta = roberta_model.roberta
        self.pos_head = pos_head
        self.additional_layer = additional_layer if additional_layer else nn.Linear(768, 768)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state
        last_hidden_states = self.additional_layer(last_hidden_states)
        pos_output = self.pos_head(last_hidden_states)
        return pos_output
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ModifiedCombinedModel(pretrained_model, pos_head).to(device)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
######################   TODO 2.2   ########################
# Report best model accuarcy on validation dataset
# Accuracy below 85% will not be graded
###################### (5 points) ##########################
import torch
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Define the loss function
loss_function = CrossEntropyLoss()

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Define the number of epochs and initialize the best accuracy
epochs = 20
max_accuracy = 0.0

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch in tqdm(trainloader, desc=f'Training Epoch {epoch + 1}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(trainloader)
    print(avg_train_loss)

    # Validation
    model.eval()
    total_labels = 0
    count = 0

    with torch.no_grad():
        for batch in tqdm(validloader, desc=f'Validation Epoch {epoch + 1}'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)


            outputs = model(input_ids, attention_mask=attention_mask)
            outputs = outputs.view(-1, outputs.shape[-1])
            labels = labels.view(-1)
            max_preds = outputs.argmax(dim = 1, keepdim = True)
            correct = max_preds.squeeze(1).eq(labels)

            total_labels += correct.sum()
            count += torch.FloatTensor([labels.shape[0]]).to(device)

    accuracy = total_labels / count
    if accuracy > max_accuracy:
        max_accuracy = accuracy



Epoch 1/15: 100%|██████████| 41/41 [00:15<00:00,  2.58it/s]


Epoch 1/15 - Average Loss: 1.4685104242185267, Accuracy: 59.84%


Epoch 1/15 - Validation: 100%|██████████| 11/11 [00:01<00:00,  8.66it/s]


Epoch 1/15 - Validation Loss: 1.027351818301461, Validation Accuracy: 70.87%


Epoch 2/15: 100%|██████████| 41/41 [00:15<00:00,  2.73it/s]


...

Epoch 14/15 - Validation Loss: 1.5302825732664629, Validation Accuracy: 50.78%


Epoch 15/15: 100%|██████████| 41/41 [00:16<00:00,  2.54it/s]


Epoch 15/15 - Average Loss: 1.5433917103744135, Accuracy: 50.59%


Epoch 15/15 - Validation: 100%|██████████| 11/11 [00:01<00:00,  8.08it/s]

Epoch 15/15 - Validation Loss: 1.563565969467163, Validation Accuracy: 49.61%


In [ ]:
######################   TODO 2.2   ########################
# Report best model accuarcy on validation dataset
# Accuracy below 85% will not be graded
# accuracy_valid
model.eval()
total_valid_correct = 0
total_valid_samples = 0
total_valid_loss = 0

with torch.no_grad():
    t = tqdm(testloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation")

    for valid_batch in t:
        input_ids_valid, attention_mask_valid, labels_valid = valid_batch['input_ids'], valid_batch['attention_mask'], valid_batch['labels']

        input_ids_valid, attention_mask_valid, labels_valid = input_ids_valid.to(device), attention_mask_valid.to(device), labels_valid.to(device)

        valid_outputs = model(input_ids_valid, attention_mask=attention_mask_valid, labels=labels_valid)
        valid_logits = valid_outputs.logits

        # Calculate the loss
        valid_loss = F.cross_entropy(valid_logits.view(-1, model.config.num_labels), labels_valid.view(-1), ignore_index=tokenizer.pad_token_id)

        total_valid_loss += valid_loss.item()

        # Calculate accuracy
        _, predicted_labels_valid = torch.max(valid_logits, 2)
        correct_valid = (predicted_labels_valid == labels_valid).sum().item()
        total_valid_correct += correct_valid
        total_valid_samples += labels_valid.numel()

average_valid_loss = total_valid_loss / len(validloader)
accuracy_valid = total_valid_correct / total_valid_samples
print(f"Epoch {epoch + 1}/{num_epochs} - Validation Loss: {average_valid_loss}, Validation Accuracy: {accuracy_valid * 100:.2f}%")

###################### (5 points) ##########################

Epoch 15/15 - Validation: 100%|██████████| 7/7 [00:00<00:00,  8.31it/s]

Epoch 15/15 - Validation Loss: 1.0118747407739812, Validation Accuracy: 49.71%


In [ ]:
######################   TODO 2.1   ########################
# Now implement it with huggingface trainer
###################### (40 points) ##########################
from transformers import AdamW, Trainer, TrainingArguments

# Define training parameters
train_params = TrainingArguments(
    output_dir='./output',
    num_train_epochs=25,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./log_files',
)

# Initialize the trainer
training_instance = Trainer(
    model=model,
    args=train_params,
    train_dataset=train,
    eval_dataset=validation,
)

# Train the model
training_result = training_instance.train()


Step,Training Loss,Validation Loss
50,1.559600,1.678694
100,1.600600,1.720884
150,1.593600,1.760777
200,1.628100,1.792430
250,1.651300,1.825404
300,1.666200,1.812489
350,1.679500,1.869642
400,1.685800,1.865487


TrainOutput(global_step=410, training_loss=1.634471265281119, metrics={'train_runtime': 180.3242, 'train_samples_per_second': 145.183, 'train_steps_per_second': 2.274, 'total_flos': 534559149062400.0, 'train_loss': 1.634471265281119, 'epoch': 10.0})